# Tools and Memory in LangChain

This notebook covers:

1. **Tools** - Allowing agents to 'act' in the real world
2. **MCP (Model Context Protocol)** - Connecting to external tool servers
3. **Memory** - Persisting state between agent invocations

---

## Setup

In [1]:
%pip install -qU langchain langchain-openai langchain-community langgraph langchain-mcp-adapters

Error processing line 1 of /Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.12/site-packages/distutils-precedence.pth:

  Traceback (most recent call last):
    File "<frozen site>", line 206, in addpackage
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named '_distutils_hack'

Remainder of file ignored
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

---

# Part 1: Tools

Tools allow agents to interact with the real world - executing code, querying databases, calling APIs, and more.

LangChain supports many tool formats. The `@tool` decorator is the most common way to create tools.

## 1.1 Basic Tool Creation

The docstring and type hints are used by the LLM to determine when and how to call the tool:

In [3]:
from typing import Literal
from langchain_core.tools import tool

@tool
def calculator(
    a: float, 
    b: float, 
    operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two numbers."""
    print(f"Calculator: {a} {operation} {b}")
    
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[calculator],
    system_prompt="You are a helpful math assistant. Use the calculator tool for calculations.",
)

In [5]:
# The agent will use the calculator tool
result = agent.invoke(
    {"messages": "What is 3.1415 * 2.7182?"}
)
print(result["messages"][-1].content)

Calculator: 3.1415 multiply 2.7182
The result of \( 3.1415 \times 2.7182 \) is approximately \( 8.5392 \).


In [6]:
# For simple math, the agent might not use the tool
result = agent.invoke(
    {"messages": "What is 3 + 4?"}
)
print(result["messages"][-1].content)

Calculator: 3.0 add 4.0
3 + 4 equals 7.


## 1.3 Multiple Tools

Agents can use multiple tools and decide which ones to call:

In [9]:
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool  
def get_random_number(min_val: int, max_val: int) -> int:
    """Generate a random integer between min_val and max_val (inclusive)."""
    import random
    return random.randint(min_val, max_val)

@tool
def reverse_string(text: str) -> str:
    """Reverse the characters in a string."""
    return text[::-1]

multi_tool_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_current_time, get_random_number, reverse_string],
    system_prompt="You are a helpful assistant with access to various tools.",
)

In [10]:
result = multi_tool_agent.invoke(
    {"messages": "What time is it, and give me a random number between 1 and 100?"}
)
print(result["messages"][-1].content)

The current time is 14:11:01, and the random number generated is 83.


In [11]:
result = multi_tool_agent.invoke(
    {"messages": "Reverse the word 'LangChain' and then tell me how many characters it has."}
)
print(result["messages"][-1].content)

The reversed word 'LangChain' is 'niahCgnaL', and it has 9 characters.


### Try Your Own Tool

Create a tool below and test it:

In [12]:
@tool
def your_custom_tool(input_param: str) -> str:
    """Describe what your tool does here.
    
    Args:
        input_param: Description of the input.
    
    Returns:
        Description of the output.
    """
    # Your implementation here
    return f"Processed: {input_param}"

# Test your tool
test_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[your_custom_tool],
)

result = test_agent.invoke({"messages": "Use the custom tool with 'hello world'"})
print(result["messages"][-1].content)

The custom tool processed your input and the result is: **Processed: hello world**.


---

# Part 2: Memory - Persisting State

By default, agents don't remember previous conversations. Memory allows you to persist messages between invocations, enabling multi-turn conversations.

## 2.1 The Problem: No Memory

In [14]:
from langchain_community.utilities import SQLDatabase
from dataclasses import dataclass
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

db = SQLDatabase.from_uri("sqlite:///./assets-resources/Chinook.db")

@dataclass
class RuntimeContext:
    db: SQLDatabase

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite SELECT query and return results."""
    runtime = get_runtime(RuntimeContext)
    try:
        return runtime.context.db.run(query)
    except Exception as e:
        return f"Error: {e}"

SYSTEM_PROMPT = """You are a SQLite analyst. Use execute_sql for queries.
Limit to 5 rows unless asked otherwise."""

# Agent WITHOUT memory
agent_no_memory = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

In [15]:
# First question
for step in agent_no_memory.stream(
    {"messages": "This is Frank Harris. What was my last invoice total?"},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

This is Frank Harris. What was my last invoice total?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_MfchCi6tg1W70MZjcnVD8RzZ)
 Call ID: call_MfchCi6tg1W70MZjcnVD8RzZ
  Args:
    query: SELECT total_amount FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT total_amount FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================

It seems that there is no table named "invoices" in the database. I can check for available tables to find the correct one that might contain in

In [ ]:
# Follow-up question - agent doesn't remember who we are!
for step in agent_no_memory.stream(
    {"messages": "What were the track titles on that invoice?"},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 3.2 The Solution: Add Memory

Use `InMemorySaver` as a checkpointer to persist conversation state:

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# Agent WITH memory
agent_with_memory = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),  # <-- This enables memory!
)

In [ ]:
# First question - use a thread_id to track the conversation
config = {"configurable": {"thread_id": "frank-session-1"}}

for step in agent_with_memory.stream(
    {"messages": "This is Frank Harris. What was my last invoice total?"},
    config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Follow-up question - now the agent remembers!
for step in agent_with_memory.stream(
    {"messages": "What were the track titles on that invoice?"},
    config,  # Same thread_id
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Another follow-up
for step in agent_with_memory.stream(
    {"messages": "What's the total price of those tracks?"},
    config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 3.3 Multiple Conversations with Thread IDs

Different `thread_id` values create separate conversation histories:

In [ ]:
# Start a NEW conversation with a different thread_id
new_config = {"configurable": {"thread_id": "another-user"}}

for step in agent_with_memory.stream(
    {"messages": "I'm a new user. What tables are in the database?"},
    new_config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Go back to Frank's conversation - it still remembers
for step in agent_with_memory.stream(
    {"messages": "Remind me, what was my name again?"},
    config,  # Back to Frank's thread_id
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Try Your Own Conversation

Have a multi-turn conversation with the agent:

In [ ]:
my_config = {"configurable": {"thread_id": "my-conversation"}}

question = "What are the most popular genres in the database?"

for step in agent_with_memory.stream(
    {"messages": question},
    my_config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

---

## Summary

In this notebook, we covered:

1. **Tools** - Extending agent capabilities:
   - `@tool` decorator for creating tools
   - Type hints and docstrings guide the LLM
   - Enhanced descriptions with `parse_docstring=True`
   - Multiple tools in a single agent

2. **MCP** - External tool servers:
   - `MultiServerMCPClient` for connecting to MCP servers
   - Standard protocol for tool interoperability

3. **Memory** - Persisting state:
   - `InMemorySaver` as a checkpointer
   - `thread_id` for tracking conversations
   - Multi-turn conversations with context

---

**Next:** [Notebook 3: Advanced Patterns](./3.0-advanced-patterns.ipynb)